In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install cvlib

In [ ]:
import torch
from torchvision import models
import torch.nn as nn

path = '/content/drive/MyDrive/ColabNotebooks/pokemonmodel_0.905.pt'

resnet50 = models.resnet50(pretrained = True)
for param in resnet50.parameters():   #가중치 freeze
  param.requires_grad = False

num_classes = 150
num_ftrs = resnet50.fc.in_features
resnet50.fc = nn.Linear(num_ftrs, num_classes)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
resnet50.to(device)


resnet50.load_state_dict(torch.load(path))
resnet50.eval()  #Dropout, Batchnorm 등의 기능 비활성화

In [ ]:
import cv2
import cvlib as cv
from google.colab.patches import cv2_imshow
import torchvision.transforms as transforms
import numpy as np
from PIL import Image
import os

img = cv2.imread('12341.PNG')
cv2_imshow(img)
img = Image.fromarray(img)

tf = transforms.Compose([
                         transforms.Resize((224,224)),  
                         transforms.ToTensor(),
                         transforms.Normalize((0.5,),(0.5,))
])


img = tf(img).unsqueeze(0)  #resnet50에 넣기 위해 4D로 변환 후 모델에 넣기위해 transform
img = img.to(device)  
pred = resnet50(img)
pokemon = pred.argmax(1)
pokemon = int(pokemon)

classes = os.listdir('/content/drive/MyDrive/ColabNotebooks/preprocessing/test')
classes.sort()
print(classes[pokemon])